# embed-text-doc2vec

based on https://medium.com/@mishra.thedeepak/doc2vec-simple-implementation-example-df2afbbfbad5

first, let's install some dependencies. a guide to doing this: https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/

In [1]:
# Install a conda package in the current Jupyter kernel
import sys
!conda install --yes --prefix {sys.prefix} gensim nltk

Fetching package metadata .........
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /Users/m/anaconda3/envs/parse-html:
#
gensim                    2.3.0               np113py36_0  
nltk                      3.2.4                    py36_0  


In [2]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

Let’s prepare data for training our doc2vec model

In [6]:
data_dir = '../../data/'

# our list of documents
data = []

In [10]:
import glob
txt_files = glob.glob(f"{data_dir}/*.txt")
print(len(txt_files))

141


In [11]:
for file in txt_files:
    with open(file, 'r', encoding="utf-8") as file:
        currentText = file.read()
        data.append(currentText)
        file.close()

print(len(data))

141


In [19]:
from random import randrange
random_index = randrange(len(data)-1)

# print the first 1000 characters of a random document from our corpus
print(data[random_index][0:1000])

Bubbleworld
A New Visual Information Retrieval Technique
Christopher Van Berendonck

Timothy Jacobs

Chris.Vanberendonck@defence.gov.au

Timothy.Jacobs@afit.edu

Air Force Institute of Technology
Wright-Patterson Air Force Base
Ohio 45433 USA

Abstract
Visualisation has significant advantages over traditional textual
lists for improving cognition in information retrieval. To realise
these advantages, we identify a set of cognitive principles and
usage patterns for information retrieval. We apply these
principles and patterns to the design of a prototype visual
information retrieval system, Bubbleworld. In Bubbleworld, we
apply a variety of visual techniques that successfully transform
the internal mental representations of the information retrieval
problem to an efficient external view and, through visual cues,
provide cognitive amplification at key stages of the information
retrieval process. We enhance the knowledge acquisition
process by providing query refinement and interaction
te

In [20]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), 
    tags=[str(i)]) for i, _d in enumerate(data)]

Here we have a list of four sentences as training data. Now I have tagged the data and its ready for training. Lets start training our model.

In [21]:
max_epochs = 100
vec_size = 20
alpha = 0.025
model = Doc2Vec(size=vec_size,
                alpha=alpha,
                min_alpha=0.00025,
                min_count=1,
                dm=1)

In [22]:
model.build_vocab(tagged_data)

In [ ]:
for epoch in range(max_epochs):
    print ('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    
model.save("d2v.model")
print("Model d2v.model Saved")

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18


Note: dm defines the training algorithm. If dm=1 means ‘distributed memory’ (PV-DM) and dm =0 means ‘distributed bag of words’ (PV-DBOW). Distributed Memory model preserves the word order in a document whereas Distributed Bag of words just uses the bag of words approach, which doesn’t preserve any word order.

So we have saved the model and it’s ready for implementation. Lets play with it.

In [13]:
from gensim.models.doc2vec import Doc2Vec

model= Doc2Vec.load("d2v.model")

#to find the vector of a document which is not in the training data
test_data = word_tokenize("I love chatbots".lower())
v1 = model.infer_vector(test_data)
print("V1_infer", v1)




V1_infer [-0.04641611  0.06686842 -0.16633473 -0.03932161 -0.21618296 -0.17176464
 -0.00281937 -0.05164452  0.06644855  0.1896757   0.07190223  0.14555858
  0.08542146  0.0769899  -0.16427462 -0.05566296 -0.02271955 -0.00445893
 -0.13367924 -0.03909039]


In [14]:
# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('1')
print(similar_doc)

[('3', 0.9997655153274536), ('0', 0.9997013211250305), ('2', 0.9996600151062012)]


In [15]:
# to find vector of doc in training data using tags
# or in other words printing the vector of the document 
# at index 1 in the training data
print(model.docvecs['1'])

[ 0.87429625 -0.9460634   2.01648474  0.3264634   2.64323592  2.13081336
 -0.04919869  0.78078324 -1.01873863 -2.38981891 -0.93454051 -1.7502445
 -1.24811769 -0.83962041  2.1012814   0.74003488  0.29993519  0.08836075
  1.56579375  0.39734492]
